In [1]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pymongo
import pickle
import string
import nltk
import re
import os
import sys
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings('ignore')



src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# helper functions
from d02_processing.cleaning_signatures import sorted_signatures
from d02_processing.cleaning_signatures import cleaned_signatures
from d01_utils.mongo_cursor_creator import mongo_cursor

# Load the "autoreload" extension
%load_ext autoreload

# reload modules so that as you change code in src, it gets loaded
%autoreload

%matplotlib inline

### Clean up and play with a small data set to make it work for the models

In [3]:
df = pd.read_csv('../../data/02_intermediate/cleaned_and_sorted.csv')
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df.head()

,curl_pattern,density,porosity,texture,products
0,2b,NaN,NaN,coarse,poo deva nopoo co wash suave naturals rinse ou...
1,3b,thick,NaN,NaN,since feb 09 currently using low poo tj s nour...
2,NaN,NaN,high,fine,dense normal to modified for washing and deta...
3,4a,NaN,NaN,NaN,deeplyenrichedoils com show me some love check...
4,3a,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...


## Run Models

In [23]:
products_df = pd.read_csv('../../data/02_intermediate/relevant_products.csv')
products_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
print(products_df.size)
products_df.head(20)

21840


,curl_pattern,density,porosity,texture,products,relevant_products
0,2b,NaN,NaN,coarse,poo deva nopoo co wash suave naturals rinse ou...,suave devacurl
1,4a,NaN,NaN,NaN,deeplyenrichedoils com show me some love check...,love shea seed
2,3a,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun
3,3b,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun
4,2c,thick,high,fine,mostly with some and a very strange curl wave...,suave as
5,3a,thick,high,fine,mostly with some and a very strange curl wave...,suave as
6,4a,NaN,NaN,NaN,demcalicurls hair type big chop june 2006 rel...,redken
7,2b,thin,NaN,NaN,former then modified now too lazy p type i k a...,NaN
8,3a,thin,NaN,NaN,former then modified now too lazy p type i k a...,NaN
9,NaN,thick,low,medium,hair type wavy normal elasticity,NaN


In [32]:
products_df.dropna(axis=0, how='any', subset=['relevant_products'], inplace=True)
print(products_df.shape)
products_df.head(20)

(2302, 6)


,curl_pattern,density,porosity,texture,products,relevant_products
0,2b,NaN,NaN,coarse,poo deva nopoo co wash suave naturals rinse ou...,suave devacurl
1,4a,NaN,NaN,NaN,deeplyenrichedoils com show me some love check...,love shea seed
2,3a,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun
3,3b,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun
4,2c,thick,high,fine,mostly with some and a very strange curl wave...,suave as
5,3a,thick,high,fine,mostly with some and a very strange curl wave...,suave as
6,4a,NaN,NaN,NaN,demcalicurls hair type big chop june 2006 rel...,redken
10,2b,NaN,high,fine,very hair low poo living proof restore shampo...,ouidad living
11,2c,NaN,high,fine,very hair low poo living proof restore shampo...,ouidad living
12,2c,medium,high,fine,wavy fii on the fia system current products l...,nexxus suave system jessicurl


In [22]:
products_df.relevant_products.value_counts()

not                                    100
love                                    87
suave                                   84
as                                      71
shea                                    62
kinky                                   47
grow                                    44
mixed                                   43
giovanni                                36
devacurl                                32
tresemme                                26
every                                   24
jessicurl                               21
shea suave                              21
aussie                                  19
herbal                                  17
la                                      16
got                                     15
ag                                      15
la suave                                15
suave aussie                            14
shea as                                 13
american                                13
vo5        

In [18]:

#Using sklearn.feature_extraction.text.CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

df = pd.DataFrame({'text': [['hello', 'world'], 
                        ['hello', 'stackoverflow', 'hello']]
                   })

## Join words to a single line as required by CountVectorizer
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x]))

In [45]:
x

<2302x79 sparse matrix of type '<class 'numpy.int64'>'
	with 4949 stored elements in Compressed Sparse Row format>

In [31]:
products_df.shape

(2302, 6)

In [48]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(products_df['relevant_products'].values)
list_of_vectors= x.toarray()
print(vectorizer.get_feature_names())
print(list_of_vectors[0])


['ag', 'alterna', 'american', 'aquage', 'as', 'aussie', 'biosilk', 'briogeo', 'bumble', 'cake', 'carol', 'curlsmith', 'design', 'devacurl', 'every', 'fairy', 'flax', 'frank', 'frederick', 'fruit', 'giovanni', 'got', 'grow', 'hask', 'herbal', 'house', 'jack', 'jessicurl', 'john', 'joico', 'kenra', 'keratin', 'kiehl', 'kinky', 'klorane', 'la', 'lime', 'living', 'loreal', 'love', 'macadamia', 'mane', 'marc', 'matrix', 'maui', 'mixed', 'mizani', 'nexxus', 'nioxin', 'not', 'noughty', 'ogx', 'ouidad', 'paul', 'philosophy', 'phyto', 'pravana', 'pura', 'pureology', 'redken', 'rusk', 'sebastian', 'seed', 'sexy', 'shea', 'sheamoisture', 'shimmer', 'soap', 'socozy', 'suave', 'sun', 'system', 'tgin', 'tigi', 'tresemme', 'ulta', 'vo5', 'wella', 'yes']
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0]


In [53]:
vector_list = []
for x in range(len(list_of_vectors)):
    vector_list.append(str(list_of_vectors[x]))
vector_df = pd.DataFrame(vector_list)
vector_df.head()

,0
0,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
1,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
4,[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [56]:
products_with_vecs = pd.concat([products_df,vector_df], ignore_index=True, axis=1)
products_with_vecs.head()

,0,1,2,3,4,5,6
0,2b,NaN,NaN,coarse,poo deva nopoo co wash suave naturals rinse ou...,suave devacurl,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
1,4a,NaN,NaN,NaN,deeplyenrichedoils com show me some love check...,love shea seed,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,3a,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,3b,NaN,NaN,NaN,hair type and lots of it hair color ranges fro...,sun,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
4,2c,thick,high,fine,mostly with some and a very strange curl wave...,suave as,[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
